In [176]:
import numpy as np
import pandas as pd
import re

dict_age = {"1": "Under 18",
            "18": "18-24",
            "25": "25-34",
            "35": "35-44",
            "45": "45-49",
            "50": "50-55",
            "56": "56+"
           }
dict_gender = {"M": "Male",
               "F": "Female"
              }
dict_occupation = {
    "0": "other or not specified",
    "1": "academic/educator",
    "2": "artist",
    "3": "clerical/admin",
    "4": "college/grad student",
    "5": "customer service",
    "6": "doctor/health care",
    "7": "executive/managerial",
    "8": "farmer",
    "9": "homemaker",
    "10": "K-12 student",
    "11": "lawyer",
    "12": "programmer",
    "13": "retired",
    "14": "sales/marketing",
    "15": "scientist",
    "16": "self-employed",
    "17": "technician/engineer",
    "18": "tradesman/craftsman",
    "19": "unemployed",
    "20": "writer"
}
data = []
with open("./movies.txt", "r") as f1:
    for line in f1:
        items = line.split("::")
        year = re.search("\((\d{4})\)", items[1]).group()
        items[1] = items[1].replace(" " + year, "")
        items.append(year[1:-1])
        items[2] = items[2].replace("\n", "").split("|")
        data.append(items)
    df_movie = pd.DataFrame(data, columns =['MovieID', 'Title', 'Genre', 'Year'])
         
data = []
with open("./users.txt", "r") as f1:
    for line in f1:
        items = line.split("::")
        items[1] = dict_gender[items[1]]
        items[2] = dict_age[items[2]]
        items[3] = dict_occupation[items[3]]
        items[4] = items[4].replace("\n", "")
        data.append(items)
    df_users = pd.DataFrame(data, columns =["UserID", "Gender", "Age", "Occupation", "Zip-code"])

data = []
with open("./training.txt", "r") as f1:
    for line in f1:
        items = line.split(",")
        items[3] = items[3].replace("\n", "")
        data.append(items)
    df_training = pd.DataFrame(data, columns =["UserID", "MovieID", "Rating", "Timestamp"])

data = []
with open("./testing.txt", "r") as f1:
    for line in f1:
        items = line.split(",")
        items[2] = items[2].replace("\n", "")
        data.append(items)
    df_testing = pd.DataFrame(data, columns =["UserID", "MovieID", "Timestamp"])

In [177]:
df_movie

,MovieID,Title,Genre,Year
0,1,Toy Story,"[Animation, Children's, Comedy]",1995
1,2,Jumanji,"[Adventure, Children's, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama]",1995
4,5,Father of the Bride Part II,[Comedy],1995
...,...,...,...,...
3878,3948,Meet the Parents,[Comedy],2000
3879,3949,Requiem for a Dream,[Drama],2000
3880,3950,Tigerland,[Drama],2000
3881,3951,Two Family House,[Drama],2000


In [178]:
df_users

,UserID,Gender,Age,Occupation,Zip-code
0,1,Female,Under 18,K-12 student,48067
1,2,Male,56+,self-employed,70072
2,3,Male,25-34,scientist,55117
3,4,Male,45-49,executive/managerial,02460
4,5,Male,25-34,writer,55455
...,...,...,...,...,...
6035,6036,Female,25-34,scientist,32603
6036,6037,Female,45-49,academic/educator,76006
6037,6038,Female,56+,academic/educator,14706
6038,6039,Female,45-49,other or not specified,01060


In [179]:
df_training

,UserID,MovieID,Rating,Timestamp
0,63,3006,4,977899676
1,1125,10,2,975625890
2,181,2384,2,977088920
3,1968,1610,4,974742431
4,4156,2410,4,965342309
...,...,...,...,...
700141,4346,3301,1,965682638
700142,1645,2359,5,988273128
700143,2719,2871,2,973285311
700144,4666,2335,3,963846717


In [180]:
df_testing

,UserID,MovieID,Timestamp
0,324,2803,976406738
1,4447,1920,965146127
2,4238,3676,965417632
3,4661,368,963867813
4,5078,3471,962416423
...,...,...,...
300058,3836,3138,965926550
300059,3632,3203,966536219
300060,3242,1627,968347685
300061,4362,2542,965188223


In [181]:
df_movie = pd.concat([df_movie,pd.get_dummies(df_movie['Genre'].apply(pd.Series).stack()).sum(level=0)],axis=1)
df_movie.drop('Genre',axis=1,inplace=True)
df_movie

,MovieID,Title,Year,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story,1995,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji,1995,0,1,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3878,3948,Meet the Parents,2000,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3879,3949,Requiem for a Dream,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3880,3950,Tigerland,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3881,3951,Two Family House,2000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [182]:
result = set(map(int, df_movie['MovieID'].unique().tolist()))
result

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 18

In [183]:
movies = set(range(1, 3593))
movies - result

{91,
 221,
 323,
 622,
 646,
 677,
 686,
 689,
 740,
 817,
 883,
 995,
 1048,
 1072,
 1074,
 1182,
 1195,
 1229,
 1239,
 1338,
 1402,
 1403,
 1418,
 1435,
 1451,
 1452,
 1469,
 1478,
 1481,
 1491,
 1492,
 1505,
 1506,
 1512,
 1521,
 1530,
 1536,
 1540,
 1560,
 1576,
 1607,
 1618,
 1634,
 1637,
 1638,
 1691,
 1700,
 1712,
 1736,
 1737,
 1745,
 1751,
 1761,
 1763,
 1766,
 1775,
 1778,
 1786,
 1790,
 1800,
 1802,
 1803,
 1808,
 1813,
 1818,
 1823,
 1828,
 1838}

In [184]:
df_zipcodes = pd.read_csv("uszips.csv")
df_zipcodes

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,all_county_weights,imprecise,military,timezone
0,601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,18570,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",False,False,America/Puerto_Rico
1,602,18.36073,-67.17517,Aguada,PR,Puerto Rico,True,NaN,41520,523.5,72003,Aguada,{'72003':100},False,False,America/Puerto_Rico
2,603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,True,NaN,54689,667.9,72005,Aguadilla,{'72005':100},False,False,America/Puerto_Rico
3,606,18.16724,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6615,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",False,False,America/Puerto_Rico
4,610,18.29032,-67.12243,Anasco,PR,Puerto Rico,True,NaN,29016,312.0,72011,Añasco,"{'72003':0.55,'72011':99.45}",False,False,America/Puerto_Rico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33094,99923,55.97796,-130.03671,Hyder,AK,Alaska,True,NaN,87,2.1,2198,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka
33095,99925,55.55767,-132.97627,Klawock,AK,Alaska,True,NaN,819,5.7,2198,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka
33096,99926,55.12617,-131.48928,Metlakatla,AK,Alaska,True,NaN,1460,4.2,2198,Prince of Wales-Hyder,{'02198':100},False,False,America/Metlakatla
33097,99927,56.25100,-133.37571,Point Baker,AK,Alaska,True,NaN,94,0.2,2198,Prince of Wales-Hyder,{'02198':100},False,False,America/Sitka


In [185]:
df_zipcodes[["zip"]] = df_zipcodes[["zip"]].applymap(str)
df_zipcodes[["city_name"]] = df_zipcodes[["city"]]
df_zipcodes.drop('city',axis=1,inplace=True)
df_users["Zip-code"] = df_users["Zip-code"].str.lstrip("0")
df_users

,UserID,Gender,Age,Occupation,Zip-code
0,1,Female,Under 18,K-12 student,48067
1,2,Male,56+,self-employed,70072
2,3,Male,25-34,scientist,55117
3,4,Male,45-49,executive/managerial,2460
4,5,Male,25-34,writer,55455
...,...,...,...,...,...
6035,6036,Female,25-34,scientist,32603
6036,6037,Female,45-49,academic/educator,76006
6037,6038,Female,56+,academic/educator,14706
6038,6039,Female,45-49,other or not specified,1060


In [186]:
df_zipcodes2 = pd.read_json('USCities.json')
df_zipcodes2

,zip_code,latitude,longitude,city,state,county
0,501,40.9223,-72.6371,Holtsville,NY,Suffolk
1,544,40.9223,-72.6371,Holtsville,NY,Suffolk
2,601,18.1653,-66.7226,Adjuntas,PR,Adjuntas
3,602,18.3931,-67.181,Aguada,PR,Aguada
4,603,18.4559,-67.1458,Aguadilla,PR,Aguadilla
...,...,...,...,...,...,...
42736,99926,55.0943,-131.567,Metlakatla,AK,Prince Wales Ketchikan
42737,99927,55.5179,-132.003,Point Baker,AK,Prince Wales Ketchikan
42738,99928,55.3954,-131.675,Ward Cove,AK,Ketchikan Gateway
42739,99929,56.4499,-132.364,Wrangell,AK,Wrangell Petersburg


In [187]:
states = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
df_zipcodes2['state'] = df_zipcodes2['state'].map(states).fillna(df_zipcodes2['state'])

In [188]:
typo_county = {
    "Du Page" : "DuPage",
    "Saint Marys" : "St. Mary's",
    "Saint Lucie" : "St. Lucie",
    "Saint Tammany" : "St. Tammany",
    "Saint Louis" : "St. Louis",
    "District Of Columbia" : "District of Columbia",
    "Saint Croix": "St. Croix",
    "De Kalb" : "DeKalb",
    "Saint Clair" : "St. Clair",
    "La Salle" : "LaSalle",
    "Saint Bernard" : "St. Bernard",
    "St Joseph": "St. Joseph",
    "Bayamon" : "Bayamón",
    "Lewis And Clark" : "Lewis and Clark",
    "Dona Ana" : "Doña Ana",
    "Saint Joseph" : "St. Joseph",
    "Saint Johns" : "St. Johns",
    "Saint Charles" : "St. Charles",
    "Saint Louis City" : "St. Louis",
    "St Joseph" : "St. Joseph",
    "Saint Lawrence" : "St. Lawrence",
    "La Porte" : "LaPorte",
    "Fond Du Lac" : "Fond du Lac"
}
typo_city = {
    "Mc Kinney" : "McKinney",
    "Mckinney" : "McKinney",
    "Bayamon" : "Bayamón",
    "W Harford": "West Harford"
}
df_zipcodes2['county'] = df_zipcodes2['county'].map(typo_county).fillna(df_zipcodes2['county'])
df_zipcodes2['city'] = df_zipcodes2['city'].map(typo_city).fillna(df_zipcodes2['city'])
df_zipcodes['city_name'] = df_zipcodes['city_name'].map(typo_city).fillna(df_zipcodes['city_name'])

In [189]:
df_users = pd.merge(df_users,df_zipcodes[['zip', 'city_name', 'state_name', 'county_name']], how='left',left_on=['Zip-code'],right_on=['zip'])
df_zipcodes2[["zip_code"]] = df_zipcodes2[["zip_code"]].applymap(str)
df_users = pd.merge(df_users,df_zipcodes2[['zip_code', 'city', 'state', 'county']], how='left',left_on=['Zip-code'],right_on=['zip_code'])
df_users

,UserID,Gender,Age,Occupation,Zip-code,zip,city_name,state_name,county_name,zip_code,city,state,county
0,1,Female,Under 18,K-12 student,48067,48067,Royal Oak,Michigan,Oakland,48067,Royal Oak,Michigan,Oakland
1,2,Male,56+,self-employed,70072,70072,Marrero,Louisiana,Jefferson,70072,Marrero,Louisiana,Jefferson
2,3,Male,25-34,scientist,55117,55117,Saint Paul,Minnesota,Ramsey,55117,Saint Paul,Minnesota,Ramsey
3,4,Male,45-49,executive/managerial,2460,2460,Newtonville,Massachusetts,Middlesex,2460,Newtonville,Massachusetts,Middlesex
4,5,Male,25-34,writer,55455,55455,Minneapolis,Minnesota,Hennepin,55455,Minneapolis,Minnesota,Hennepin
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,6036,Female,25-34,scientist,32603,32603,Gainesville,Florida,Alachua,32603,Gainesville,Florida,Alachua
6036,6037,Female,45-49,academic/educator,76006,76006,Arlington,Texas,Tarrant,76006,Arlington,Texas,Tarrant
6037,6038,Female,56+,academic/educator,14706,14706,Allegany,New York,Cattaraugus,14706,Allegany,New York,Cattaraugus
6038,6039,Female,45-49,other or not specified,1060,1060,Northampton,Massachusetts,Hampshire,1060,Northampton,Massachusetts,Hampshire


In [190]:
df_users['same'] = np.where((df_users['city_name'] == df_users['city']) & (df_users['state_name'] <= df_users['state']) & (df_users['county_name'] <= df_users['county']), True, False)

In [191]:
#pd.set_option('display.max_rows', 204)
df_users.loc[(df_users['same']==False) & (df_users['zip'].notnull()) & (df_users['zip_code'].notnull()) & ((df_users['city_name'] != df_users['city']) | (df_users['state_name'] != df_users['state']) | (df_users['county_name'] != df_users['county']))]
# Saint Paul di Ramsey, bukan di Washington(Wikipedia & https://www.unitedstateszipcodes.org/55110/)
# Atlanta di Fulton, bukan DeKalb(Wikipedia & https://www.unitedstateszipcodes.org/30350/)
# 66212 di Overland, bukan Shawnee(meskipun bisa di-accept juga)(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/66212/)
# Frisco di Collin dan Denton(harus 2-2nya, bukan salah satu saja) (Wikipedia)
#        di Denton(https://www.unitedstateszipcodes.org/75034/)
# 66205 di Roeland Park, bukan di Mission dan Shawnee(https://www.mapdevelopers.com/what-is-my-zip-code.php)
#       di Mission(https://www.unitedstateszipcodes.org/66205/)
# 96860 tidak ada nama city-nya(https://www.mapdevelopers.com/what-is-my-zip-code.php)
#       di Jbphh, bukan Pearl Harbor(meskipun singkatannya) (https://www.unitedstateszipcodes.org/96860/)
# 53092 di Thiensville, bukan Mequon(meskipun bisa di-accept juga)(https://www.unitedstateszipcodes.org/53092/)
# 66211 di Overland, bukan Leawood dan Shawnee(https://www.mapdevelopers.com/what-is-my-zip-code.php)
#       di Leawood(https://www.unitedstateszipcodes.org/66211/)
# McKinney, bukan Mckinney atau Mc Kinney (Wikipedia)
# 75287 di Dallas, bukan Denton(Wikipedia)
#       di Denton(https://www.unitedstateszipcodes.org/75287/)
# 80020 di Broomfield, bukan Boulder(Wikipedia & https://www.unitedstateszipcodes.org/80020/)
# 2125 di Dorchester, bukan Boston(meskipun bisa di-accept juga)(https://www.unitedstateszipcodes.org/02125/)
# 60103 bisa di Cook, DuPage atau Kane(https://www.village.bartlett.il.us/our-community/counties)
#      di DuPage(https://www.unitedstateszipcodes.org/60103/)
# 2127 di South Boston, bukan di Boston(Wikipedia)
#      di Boston(di Mapnya ada Southnya(https://www.unitedstateszipcodes.org/02127/)
# 77345 di Kingwood, bukan Humble(https://www.unitedstateszipcodes.org/77345/)
# 48178 di Oakland, bukan Washtenaw(https://www.unitedstateszipcodes.org/77345/)
# 66224 di Overland Park, bukan Shawnee(meskipun bisa di-accept juga)(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/66224/)
# 66204 di Overland Park, bukan Shawnee(meskipun bisa di-accept juga)(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/66204/)
# 54915 di Outagamie, bukan Winnebago(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/54915/)
# 66208 di Prairie Village, bukan Shawnee(meskipun bisa di-accept juga)(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/66208/)
# 23059 di Henrico, bukan Hanover(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/23059/)
# 60042 di McHenry, bukan Lake(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/60042/)
# 66203 di Shawnee, bukan Shawnee Mission(meskipun bisa di-accept juga)(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/66203/)
# 80227 di Denver, bukan Jefferson(https://www.mapdevelopers.com/what-is-my-zip-code.php)
#       di Jefferson (https://www.unitedstateszipcodes.org/80227/)
# 23229 di Henrico, bukan Richmond(https://www.unitedstateszipcodes.org/23229/)
# 23233 di Henrico, bukan Richmond(https://www.unitedstateszipcodes.org/23233/)
# 24450 di Rockbridge, bukan Lexington City(https://www.unitedstateszipcodes.org/24450/)
# 19003 di Montgomery, bukan Delaware(https://www.unitedstateszipcodes.org/19003/)
# 92509 di Riverside, bukan Jurupa Valley(https://www.unitedstateszipcodes.org/92509/)
# 30662 di Franklin, bukan Madison(https://www.mapdevelopers.com/what-is-my-zip-code.php & https://www.unitedstateszipcodes.org/30662/)
# 30052 di Walton, bukan Gwinnett(https://www.unitedstateszipcodes.org/30052/)
# 80227 di Osseo, bukan Maple Grove(meskipun bisa di-accept juga)(https://www.unitedstateszipcodes.org/80227/)
# 84119 di Salt Lake City, bukan West Valley City(meskipun bisa di-accept juga)(https://www.unitedstateszipcodes.org/84119/)
# 33028 di Pembroke Pines, bukan Hollywood(https://www.unitedstateszipcodes.org/33028/)

,UserID,Gender,Age,Occupation,Zip-code,zip,city_name,state_name,county_name,zip_code,city,state,county,same
36,37,Female,25-34,homemaker,66212,66212,Overland Park,Kansas,Johnson,66212,Shawnee Mission,Kansas,Johnson,False
56,57,Male,18-24,unemployed,30350,30350,Atlanta,Georgia,Fulton,30350,Atlanta,Georgia,Dekalb,False
96,97,Female,35-44,clerical/admin,66210,66210,Overland Park,Kansas,Johnson,66210,Shawnee Mission,Kansas,Johnson,False
133,134,Male,25-34,other or not specified,66212,66212,Overland Park,Kansas,Johnson,66212,Shawnee Mission,Kansas,Johnson,False
290,291,Male,35-44,programmer,55110,55110,Saint Paul,Minnesota,Washington,55110,Saint Paul,Minnesota,Ramsey,False
343,344,Male,35-44,sales/marketing,75034,75034,Frisco,Texas,Denton,75034,Frisco,Texas,Collin,False
345,346,Female,25-34,other or not specified,55110,55110,Saint Paul,Minnesota,Washington,55110,Saint Paul,Minnesota,Ramsey,False
347,348,Male,50-55,executive/managerial,55110,55110,Saint Paul,Minnesota,Washington,55110,Saint Paul,Minnesota,Ramsey,False
368,369,Male,35-44,academic/educator,55110,55110,Saint Paul,Minnesota,Washington,55110,Saint Paul,Minnesota,Ramsey,False
381,382,Female,35-44,writer,66205,66205,Mission,Kansas,Johnson,66205,Shawnee Mission,Kansas,Johnson,False


In [192]:
df_users.loc[(df_users['same']==False) & (df_users['zip'].isnull()) & (df_users['zip_code'].isnull())]
# Sumber utama dari https://www.unitedstateszipcodes.org/, diperkuat dengan https://www.mapdevelopers.com/what-is-my-zip-code.php.
# Kalau sumber keduanya agak ngaco, percaya sumber utama saja. Kalau 2-2nya nggak jelas, boleh coba cari di Google. Kalau nggak ada, tulis nggak ada.
# Bagian ini kalau ada yang memang ada, taruh di paling atas/paling bawah biar jelas.
# 64043 tidak ada
# 98107-2117 tidak ada. 98107 itu Seattle, Washington, King.  2117 itu Boston, Massachusetts, Suffolk
# 37919-4204 tidak ada. 37919 itu Knoxville, Tennessee, Knox. 4204 itu tidak ada
# 98153 tidak ada.
# dst (Tolong dipastikan apakah ada atau tidak)

,UserID,Gender,Age,Occupation,Zip-code,zip,city_name,state_name,county_name,zip_code,city,state,county,same
142,143,Male,18-24,clerical/admin,64043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
160,161,Male,45-49,self-employed,98107-2117,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
232,233,Female,45-49,writer,37919-4204,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
234,235,Male,25-34,other or not specified,98153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
292,293,Male,56+,academic/educator,55337-4056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
335,336,Male,18-24,other or not specified,98765,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
457,458,Male,50-55,self-employed,55405-2546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
505,506,Male,25-34,self-employed,55103-1006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
540,541,Female,18-24,college/grad student,5849574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
549,550,Male,45-49,farmer,21559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [193]:
df_users.loc[(df_users['same']==False) & (df_users['zip'].isnull()) & (df_users['zip_code'].notnull())]
# Sumber utama dari https://www.unitedstateszipcodes.org/, diperkuat dengan https://www.mapdevelopers.com/what-is-my-zip-code.php.
# Kalau sumber keduanya agak ngaco, percaya sumber utama saja. Kalau 2-2nya nggak jelas, boleh coba cari di Google. Kalau nggak ada, tulis nggak ada.
# 32793 di Winter Park, Florida, Orange
# 96931 di Tamuning, Guam, Guam
# 6520 di New Haven, Connecticut, New Haven
# 21203 di Baltimore, Maryland, Baltimore City

,UserID,Gender,Age,Occupation,Zip-code,zip,city_name,state_name,county_name,zip_code,city,state,county,same
11,12,Male,25-34,programmer,32793,NaN,NaN,NaN,NaN,32793,Winter Park,Florida,Orange,False
52,53,Male,25-34,other or not specified,96931,NaN,NaN,NaN,NaN,96931,Tamuning,Guam,Guam,False
130,131,Male,18-24,college/grad student,6520,NaN,NaN,NaN,NaN,6520,New Haven,Connecticut,New Haven,False
173,174,Male,25-34,self-employed,21203,NaN,NaN,NaN,NaN,21203,Baltimore,Maryland,Baltimore City,False
295,296,Male,50-55,customer service,89432,NaN,NaN,NaN,NaN,89432,Sparks,Nevada,Washoe,False
390,391,Male,45-49,lawyer,22122,NaN,NaN,NaN,NaN,22122,Newington,Virginia,Fairfax,False
504,505,Male,35-44,technician/engineer,37815,NaN,NaN,NaN,NaN,37815,Morristown,Tennessee,Hamblen,False
568,569,Female,18-24,college/grad student,97339,NaN,NaN,NaN,NaN,97339,Corvallis,Oregon,Benton,False
662,663,Female,25-34,executive/managerial,12345,NaN,NaN,NaN,NaN,12345,Schenectady,New York,Schenectady,False
699,700,Male,50-55,technician/engineer,14650,NaN,NaN,NaN,NaN,14650,Rochester,New York,Monroe,False


In [194]:
df_users.loc[(df_users['same']==False) & (df_users['zip'].notnull()) & (df_users['zip_code'].isnull())]
# 80023 di Broomfield, Colorado, Broomfield

,UserID,Gender,Age,Occupation,Zip-code,zip,city_name,state_name,county_name,zip_code,city,state,county,same
1888,1889,Male,18-24,sales/marketing,80023,80023,Broomfield,Colorado,Broomfield,NaN,NaN,NaN,NaN,False
